In [3]:
from loading_data import *

/Users/maximeswagel/Library/Mobile Documents/com~apple~CloudDocs/Documents/EPFL/DTU/Courses/SecondSemester/MachineLearning/Assignment2/scripts

Location of the south_african_heart_disease.xls file: /Users/maximeswagel/Library/Mobile Documents/com~apple~CloudDocs/Documents/EPFL/DTU/Courses/SecondSemester/MachineLearning/Assignment2/data/south_african_heart_disease.xls


In [11]:
print(X)
print(classNames)
print(attributeNames)
print(N)
print(M)

[[1.440e+02 1.000e-02 4.410e+00 ... 2.060e+00 6.300e+01 0.000e+00]
 [1.420e+02 4.050e+00 3.380e+00 ... 2.620e+00 3.800e+01 0.000e+00]
 [2.060e+02 6.000e+00 2.950e+00 ... 5.606e+01 6.000e+01 0.000e+00]
 ...
 [1.240e+02 1.600e+00 7.220e+00 ... 0.000e+00 5.100e+01 1.000e+00]
 [1.700e+02 4.000e-01 4.110e+00 ... 2.060e+00 5.700e+01 1.000e+00]
 [1.320e+02 0.000e+00 4.820e+00 ... 0.000e+00 4.600e+01 1.000e+00]]
{'chd yes', 'chd no'}
['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age', 'famhist']
462
9


#### 1) Explain which classification problem you have chosen to solve. Is it a multi-class or binary classification problem?

The classification problem that we are solving is to predict chd (coronary heart disease) based on the attributes. It is a binary classification problem. The direct interpretation is to know if someone has had a chd based on data. We can extend this to predicting if someone will likely have a chd based on the collected data on this individual. (this is given that the person keeps that same habits) -> maybe this is not applicable because there is an age attribute.
Another use of this classification can be for an insurance company to determine if a client likely has had a chd to then be able to adjust the cost for the insurance.

#### 2. We will compare logistic regression, method 2 and a baseline. For logistic regression, we will once more use λ as a complexity-controlling parameter, and for method 2 a relevant complexity controlling parameter and range of values. We recommend this choice is made based on a trial run, which you do not need to report. Describe which parameter you have chosen and the possible values of the parameters you will examine. The baseline will be a model which compute the largest class on the training data, and predict everything in the test-data as belonging to that class (corresponding to the optimal prediction by a logistic regression model with a bias term and no features).

In [69]:
#First we want to normalize and transform our data.

from scipy.stats import zscore

normalised_X = np.copy(X)
#transform
normalised_X[:,6] = np.log(1 + X[:,6]) #add 1 because some alcohol values are 0
#normalise
normalised_X = zscore(normalised_X, axis = 0, ddof = 1)

attributeNames_norm = np.copy(attributeNames)
attributeNames_norm[6] = 'log-alc'
attributeNames_norm = ['normalized ' + attribute for attribute in attributeNames_norm]

#Or without the last binary data

Y = np.copy(normalised_X[:,:-1])
N_y, M_y = Y.shape

attributeNames_y = np.copy(attributeNames_norm[:-1])

In [76]:
print(M,N)

9 462


In [70]:
import numpy as np
from sklearn import model_selection
from sklearn.dummy import DummyClassifier
import torch
from sklearn.linear_model import LogisticRegression
from dtuimldmtools import train_neural_net

In [ ]:
#We need to consider the possible lambdas and the number of hidden units that we want to consider in the inner loop. 
#For the baseline model there is no controlling parameter

#First try
lambda_interval = np.logspace(-8, 2, 50)
n_hidden_units = np.arange(1,3)
print(n_hidden_units)

K_out = 2
K_in = 2
CV_out = model_selection.KFold(K_out,shuffle=True)
CV_in = model_selection.KFold(K_in,shuffle=True)

# For statistical evaluation : store outer fold predictions for the three models
yhat = []
y_true = []

# For debugging : store inner fold predictions (for each outer fold)
Logistic_full = np.zeros((K_in, len(lambda_interval), K_out, 2))
ANN_full = np.zeros((K_in, len(n_hidden_units), K_out, 2))
#where the last coordinate gives us if we are looking at the estimate or the validation values (0 for estimation and 1 for validation)

#The Error for the best model of each type of model in each outer loop
# Train_error = np.zeros((K_out,3))
Test_error= np.zeros((K_out,3))
best_lambda_index = np.int32(np.zeros(K_out))
best_h_index = np.int32(np.zeros(K_out))

for k, (train_index,test_index) in enumerate(CV_out.split(normalised_X,y)):
    #to store the new predictions of the selected model at each outer fold (to then be concatenated in yhat)
    dy = []

    #the training tests for each fold of the outer loop
    X_train = normalised_X[train_index, :]
    y_train = y[train_index]
    X_test = normalised_X[test_index, :]
    y_test = y[test_index]

    #Baseline model
    baseline = DummyClassifier(strategy='most_frequent')
    baseline.fit(X_train,y_train)
    Test_error[k,2] = 1-baseline.score(X_test,y_test)

    #INITIALIZE ERROR HANDLING
    #Error for each model in each of the loops, overwritten at each outer loop 
    Logistic_Inner_test_error= np.zeros((K_in, len(lambda_interval)))
    ANN_Inner_test_error= np.zeros((K_in, len(n_hidden_units)))

    #average error of each model on each outter fold
    Logistic_Model_out_test_error = np.zeros((K_out, len(lambda_interval)))
    ANN_Model_out_test_error = np.zeros((K_out, len(n_hidden_units)))

    #we also need to store the sizes of the folds
    inner_fold_validate_sizes = np.zeros(K_in)

    #Inner Loop
    for i, (Inner_train_index, Inner_test_index) in enumerate(CV_in.split(X_train,y_train)):
        #initialize the training and validation sets
        X_subtrain = X_train[Inner_train_index]
        y_subtrain = y[Inner_train_index]
        X_validate = X_train[Inner_test_index]
        y_validate = y_train[Inner_test_index]

        #store the size of the validation set
        inner_fold_validate_sizes[i] = X_validate.shape[0]

        #=========#
        #Logistic regression
        #=========#

        #Logistic Regression Model Loop (hyper_parameter tuning)
        for s,lamb in enumerate(lambda_interval):
            
            mdl = LogisticRegression(penalty="l2", C=1 / lamb)
            mdl.fit(X_subtrain, y_subtrain)

            # y_subtrain_est = mdl.predict(X_train).T
            y_validate_est = mdl.predict(X_validate).T
            Logistic_Inner_test_error[i, s] = np.sum(y_validate_est != y_validate) / len(y_validate)
            Logistic_full[i,s,k,0] = y_validate_est
            Logistic_full[i,s,k,1] = y_validate

        #==========#
        #ANN
        #==========#

        #We convert the training and test sets to torch tensors

        X_subtrain = torch.tensor(X_subtrain, dtype=torch.float32)
        y_subtrain = torch.tensor(y_subtrain, dtype=torch.float32).reshape(-1, 1)
        X_validate = torch.tensor(X_validate, dtype=torch.float32)
        y_validate = torch.tensor(y_validate, dtype=torch.float32).reshape(-1, 1)

        #ANN cross validation loop
        for j, n in enumerate(n_hidden_units):
            # The lambda-syntax defines an anonymous function, which is used here to
            # make it easy to make new networks within each cross validation fold
            model = lambda: torch.nn.Sequential(
                torch.nn.Linear(M, n),  # M features to H hiden units
                # 1st transfer function, either Tanh or ReLU:
                torch.nn.Tanh(),  # torch.nn.ReLU(),
                torch.nn.Linear(n, 1),  # H hidden units to 1 output neuron
                torch.nn.Sigmoid(),  # final tranfer function
            )
            #Loss function (Binary Cross Entropy)
            loss_fn = torch.nn.BCELoss()
            # Train for a maximum of 10000 steps, or until convergence
            max_iter = 10000
        
            net, final_loss, learning_curve = train_neural_net(
            model, loss_fn, X=X_subtrain, y=y_subtrain, n_replicates=1, max_iter=max_iter
            )
            y_validate_est = net(X_validate).detach().numpy()
            y_validate_pred = (y_validate_est > 0.5).astype(int)
            y_validate_np = y_validate.numpy()
            ANN_Inner_test_error[i, j] = np.sum(y_validate_pred != y_validate) / len(y_validate)
            ANN_full[i,j,k,0] = y_validate_pred
            ANN_full[i,j,k,1] = y_validate

    #Average Model Error calculation for Regression   
    Logistic_Model_out_test_error = np.sum(inner_fold_validate_sizes[:,None]*Logistic_Inner_test_error, axis = 0)/X_train.shape[0]
    best_lambda_index[k] = int(np.argmin(Logistic_Model_out_test_error))
    #Average Model Error calculation for ANN   
    ANN_Model_out_test_error = np.sum(inner_fold_validate_sizes[:,None]*ANN_Inner_test_error, axis = 0)/X_train.shape[0]
    best_h_index[k] = int(np.argmin(ANN_Model_out_test_error))

    #Retrain the best model on the full X_train for regression
    mdl = LogisticRegression(penalty="l2", C = 1/lambda_interval[best_lambda_index[k]])
    mdl.fit(X_train,y_train)
    y_test_est = mdl.predict(X_test).T
    Test_error[k,0] = np.sum(y_test_est != y_test) / len(y_test)
    
    #to store the predictions
    dy.append(y_test_est)

    #Retrain the best ANN model
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
    model = lambda: torch.nn.Sequential(
        torch.nn.Linear(M, n_hidden_units[best_h_index[k]]),  # M features to H hiden units
        # 1st transfer function, either Tanh or ReLU:
        torch.nn.Tanh(),  # torch.nn.ReLU(),
        torch.nn.Linear(n_hidden_units[best_h_index[k]], 1),  # H hidden units to 1 output neuron
        torch.nn.Sigmoid(),  # final tranfer function
    )
    #Loss function (Binary Cross Entropy)
    loss_fn = torch.nn.BCELoss()
    # Train for a maximum of 10000 steps, or until convergence
    max_iter = 10000
        
    net, final_loss, learning_curve = train_neural_net(
        model, loss_fn, X=X_train, y=y_train, n_replicates=1, max_iter=max_iter
    )
    y_test_est = net(X_test).detach().numpy()
    y_test_pred = (y_test_est > 0.5).astype(int)
    y_test = y_test.numpy()
    Test_error[k, 1] = np.sum(y_test_pred != y_test) / len(y_test)


    #to store the predictions
    dy.append(y_test_est)
    dy = np.stack(dy, axis=1)
    yhat.append(dy)
    y_true.append(y_test)

[ 1  2  3  4  5  6  7  8  9 10]

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.5694492	5.7568554e-06
		2000	0.56596863	6.52945e-06
		3000	0.5605413	1.03143e-05
		4000	0.55595195	6.3254597e-06
		5000	0.55313534	4.0947793e-06
		6000	0.5511564	3.1361865e-06
		7000	0.54958767	2.6028745e-06
		8000	0.54825544	2.1743338e-06
		9000	0.54709196	2.0700102e-06
		10000	0.54606956	1.637279e-06
		Final loss:
		10000	0.54606956	1.637279e-06

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.5717939	2.5955454e-05
		2000	0.5669623	4.415433e-06
		3000	0.5632857	8.99428e-06
		4000	0.5583814	7.4721224e-06
		5000	0.5548771	5.1561115e-06
		6000	0.55241764	3.7764084e-06
		7000	0.550551	2.9231092e-06
		8000	0.5490156	2.497021e-06
		9000	0.547698	2.2853737e-06
		10000	0.5465539	1.853938e-06
		Final loss:
		10000	0.5465539	1.853938e-06

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.5687613	1.4566612e-05
		2000	0.56401104	1.0039492e-05
		3000	0.5584269	8.538863e-06
		4000	0.5547256	5.050072e-06
		5000	0.5523

In [115]:
print(inner_fold_validate_sizes)
print(Logistic_Inner_test_error.shape)
print(Logistic_Inner_test_error[:,0])
print(sum(Logistic_Inner_test_error[:,0]))
print(np.sum(Logistic_Inner_test_error, axis = 0)[0])
#So the right axis to sum over is 0

[74. 74. 74. 74. 74.]
(5, 50)
[0.22972973 0.33783784 0.35135135 0.39189189 0.31081081]
1.6216216216216215
1.6216216216216215


In [108]:
print((inner_fold_validate_sizes[:,None] * Logistic_Inner_test_error)[:,0])
print(Logistic_Inner_test_error[:,0]*inner_fold_validate_sizes)
#So this broadcasting works as intended

[17. 25. 26. 29. 23.]
[17. 25. 26. 29. 23.]


In [112]:
print(ANN_Model_out_test_error)
print(Logistic_Model_out_test_error)

[0.01351351 0.01351351 0.01351351 0.01351351 0.01351351 0.01351351
 0.01351351 0.01351351 0.01351351 0.01351351]
[0.32432432 0.32432432 0.32432432 0.32432432 0.32432432 0.32432432
 0.32432432 0.32432432 0.32432432 0.32432432 0.32432432 0.32432432
 0.32432432 0.32432432 0.32432432 0.32432432 0.32432432 0.32432432
 0.32432432 0.32432432 0.32432432 0.32432432 0.32432432 0.32432432
 0.32432432 0.32432432 0.32432432 0.32432432 0.32432432 0.32432432
 0.32432432 0.32432432 0.32432432 0.32432432 0.32432432 0.32432432
 0.32432432 0.32432432 0.32432432 0.32432432 0.32432432 0.32702703
 0.32702703 0.33243243 0.34054054 0.34324324 0.34864865 0.34864865
 0.34864865 0.34864865]


In [110]:
# The lambda show that there should be no control parameter. 
# This means that the problem is inherently complicated and that there is not risk of overfitting with logistic regression.
# Maybe we can try with the logit link instead of the sigmoid function ?
print(f'optimal control parameter index : {best_lambda_index}')
print(f'optimal control parameter : {lambda_interval[best_lambda_index]}')
print(f'Optimal number of hidden units : {n_hidden_units[best_h_index]}')
print(Test_error)

optimal control parameter index : [41  0 48 40  0]
optimal control parameter : [2.32995181e+00 1.00000000e-08 6.25055193e+01 1.45634848e+00
 1.00000000e-08]
Optimal number of hidden units : [1 1 1 1 1]
[[0.27956989 0.27956989 0.30107527]
 [0.27956989 0.27956989 0.43010753]
 [0.32608696 0.36956522 0.31521739]
 [0.26086957 0.26086957 0.34782609]
 [0.23913043 0.25       0.33695652]]


In [ ]:
#test the length of the folds : 
K_out = 5
K_in = 5
CV_out = model_selection.KFold(K_out,shuffle=True)
CV_in = model_selection.KFold(K_in,shuffle=True)

#The Error for the best model of each type of model in each outer loop
Train_error = np.zeros((K_out,3))
Test_error= np.zeros((K_out,3))

for k, (train_index,test_index) in enumerate(CV_out.split(normalised_X,y)):
    #the training tests for each fold of the outer loop
    X_train = normalised_X[train_index, :]
    y_train = y[train_index]
    X_test = normalised_X[test_index, :]
    y_test = y[test_index]
    print(X_train.shape)

#Conclusion : we need to record them

(369, 9)
(369, 9)
(370, 9)
(370, 9)
(370, 9)


We go on with the rest and then come back on this issue afterwards

Perform a statistical evaluation of your three models similar to the previous section. That
is, compare the three models pairwise. We will once more allow some freedom in what test
to choose. Therefore, choose either:
setup I (section 11.3): Use McNemar’s test described in Box 11.3.2)
setup II (section 11.4): Use the method described in Box 11.4.1)
Include p-values and confidence intervals for the three pairwise tests in your report and
conclude on the results: Is one model better than the other? Are the two models better
than the baseline? Are some of the models identical? What recommendations would you
make based on what you’ve learned?

In [114]:
from dtuimldmtools import mcnemar

In [ ]:
#we suppose that we have the y_hat and y_true for all models.

# Compute the Jeffreys interval
alpha = 0.05
[thetahat, CI, p] = mcnemar(y_true, yhat[:, 0], yhat[:, 2], alpha=alpha)